In [1]:
import requests
import pandas as pd
import mysql.connector
from datetime import datetime

DB_USER = 'root'
DB_PASS = 'root'
DB_HOST = 'localhost'
DB_PORT = '3306'
DB_NAME = 'airquality'
TABLE_NAME = 'airquality1'

cities = {
    "Delhi": (28.6519, 77.2315),
    "Mumbai": (19.0728, 72.8826),
    "Bengaluru": (12.9719, 77.5937),
    "Kolkata": (22.5626, 88.363),
    "Chennai": (13.0878, 80.2785),
    "Hyderabad": (17.384, 78.4564),
    "Bhopal": (22.7179, 75.8333),
    "Mysuru": (12.2979, 76.6393),
    "Mandi": (31.7119, 76.9327),
    "Shillong": (25.5689, 91.8831),
    "Gandhinagar": (23.2167, 72.6833),
    "Amritsar": (31.6223, 74.8753)
}

conn = mysql.connector.connect(
    host=DB_HOST,
    user=DB_USER,
    password=DB_PASS
)
cursor = conn.cursor()
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {DB_NAME}")
cursor.execute(f"USE {DB_NAME}")

# --- Create Table (updated with methane) ---
create_table_sql = f'''
CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
    city VARCHAR(50),
    time DATETIME,
    pm10 FLOAT,
    pm2_5 FLOAT,
    carbon_monoxide FLOAT,
    carbon_dioxide FLOAT,
    nitrogen_dioxide FLOAT,
    sulphur_dioxide FLOAT,
    ozone FLOAT,
    dust FLOAT,
    uv_index FLOAT,
    methane FLOAT,
    PRIMARY KEY (city, time)
)
'''
cursor.execute(create_table_sql)
conn.commit()

base_url = "https://air-quality-api.open-meteo.com/v1/air-quality"
parameters = [
    "pm10", "pm2_5", "carbon_monoxide", "carbon_dioxide",
    "nitrogen_dioxide", "sulphur_dioxide", "ozone", "dust", "uv_index", "methane"
]

for city, (lat, lon) in cities.items():
    params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": ",".join(parameters),
        "timezone": "auto",
        "past_days": 5,
        "forecast_days": 3,
        "temporal_resolution": "hourly_6"
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    if "hourly" not in data:
        print(f"❌ No data for {city}")
        continue

    hourly_data = data["hourly"]
    times = pd.to_datetime(hourly_data["time"])

    for i, time in enumerate(times):
        record = (
            city,
            time.strftime("%Y-%m-%d %H:%M:%S"),
            hourly_data.get("pm10", [None])[i],
            hourly_data.get("pm2_5", [None])[i],
            hourly_data.get("carbon_monoxide", [None])[i],
            hourly_data.get("carbon_dioxide", [None])[i],
            hourly_data.get("nitrogen_dioxide", [None])[i],
            hourly_data.get("sulphur_dioxide", [None])[i],
            hourly_data.get("ozone", [None])[i],
            hourly_data.get("dust", [None])[i],
            hourly_data.get("uv_index", [None])[i],
            hourly_data.get("methane", [None])[i]
        )

        insert_sql = f'''
        INSERT INTO {TABLE_NAME} (
            city, time, pm10, pm2_5, carbon_monoxide, carbon_dioxide,
            nitrogen_dioxide, sulphur_dioxide, ozone, dust, uv_index, methane
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
            pm10=VALUES(pm10), pm2_5=VALUES(pm2_5), carbon_monoxide=VALUES(carbon_monoxide),
            carbon_dioxide=VALUES(carbon_dioxide), nitrogen_dioxide=VALUES(nitrogen_dioxide),
            sulphur_dioxide=VALUES(sulphur_dioxide), ozone=VALUES(ozone),
            dust=VALUES(dust), uv_index=VALUES(uv_index), methane=VALUES(methane)
        '''
        cursor.execute(insert_sql, record)

    conn.commit()
    print(f"✅ Inserted data for {city}")

cursor.close()
conn.close()
print("🎉 All city data inserted successfully!")


✅ Inserted data for Delhi
✅ Inserted data for Mumbai
✅ Inserted data for Bengaluru
✅ Inserted data for Kolkata
✅ Inserted data for Chennai
✅ Inserted data for Hyderabad
✅ Inserted data for Bhopal
✅ Inserted data for Mysuru
✅ Inserted data for Mandi
✅ Inserted data for Shillong
✅ Inserted data for Gandhinagar
✅ Inserted data for Amritsar
🎉 All city data inserted successfully!
